### Duplicate Record 핸들링을 위한 파이썬 함수
    - 앞선 복잡하게 insert했던것들 해결

In [5]:
import pickle

file = open("/Users/kangyeolyoun/Desktop/pickle file/aws_mysql_db", "rb")
data = pickle.load(file)

In [3]:
def get_headers(client_id, client_secret):

    endpoint = "https://accounts.spotify.com/api/token"
    encoded = base64.b64encode("{}:{}".format(client_id, client_secret).encode('utf-8')).decode('ascii')

    headers = {
        "Authorization": "Basic {}".format(encoded)
    }

    payload = {
        "grant_type": "client_credentials"
    }

    r = requests.post(endpoint, data=payload, headers=headers)

    access_token = json.loads(r.text)['access_token']

    headers = {
        "Authorization": "Bearer {}".format(access_token)
    }

    return headers

def insert_row(cursor, data, table):
    placeholders = ", ".join(["%s"] * len(data))
    columns = ", ".join(data.keys())
    key_placeholders = ", ".join(["{0}=%s".format(k) for k in data.keys()]) #id = %s, name=%s 이런식
    sql = "INSERT INTO %s (%s) VALUES (%s) ON DUPLICATE KEY UPDATE %s" % (table, columns, placeholders,\
                                                                         key_placeholders)
    print(sql)
    sys.exit(0)
    cursor.execute(sql, list(data.values())*2)

In [10]:
# 원하는 데이터 뽑아오자
import sys
import requests
import base64
import json
import logging
import pymysql

host = data[0]
port = int(data[1])
username = "kangyeol"
database = data[2]
password = data[3]
client_id = data[4]
client_secret = data[5]


def main():
    
    
    
    try:
        conn = pymysql.connect(host, user=username, passwd=password, db=database,\
                             port=port, use_unicode=True, charset="utf8")
        cursor = conn.cursor()
    except:
        logging.error("could not connect to RDS")
        sys.exit(1)
        
    headers = get_headers(client_id, client_secret)
    
    ## Spotify Search API
    params = {
        "q": "BTS",
        "type": "artist",
        "limit": "5"
    }
    
    r = requests.get("https://api.spotify.com/v1/search", params=params, headers=headers)
    
    # json을 통해서 스트링 형태로 들어온다.
    raw = json.loads(r.text)

    artist = {}
    artist_raw = raw["artists"]["items"][0]
    
    if artist_raw["name"] == params["q"]:
        artist.update({
            "id": artist_raw["id"],
            "name": artist_raw["name"],
            "followers": artist_raw["followers"]["total"],
            "popularity": artist_raw["popularity"],
            "url": artist_raw["external_urls"]["spotify"],
            "image_url": artist_raw["images"][0]["url"]
        })
    
    insert_row(cursor, artist, "artists")
    conn.commit()
    sys.exit(0)
    
    


In [11]:
main()

INSERT INTO artists (id, name, followers, popularity, url, image_url) VALUES (%s, %s, %s, %s, %s, %s) ON DUPLICATE KEY UPDATE id=%s, name=%s, followers=%s, popularity=%s, url=%s, image_url=%s


SystemExit: 0

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# 이런식으로 values와 Douplicate 안에 %s로 입력되도록 만들고,

In [12]:
# insert into duplicate 간단화
def insert_row(cursor, data, table):
    placeholders = ", ".join(["%s"] * len(data))
    columns = ", ".join(data.keys())
    key_placeholders = ", ".join(["{0}=%s".format(k) for k in data.keys()]) #id = %s, name=%s 이런식
    sql = "INSERT INTO %s (%s) VALUES (%s) ON DUPLICATE KEY UPDATE %s" % (table, columns, placeholders,\
                                                                         key_placeholders)
    cursor.execute(sql, list(data.values())*2)

In [13]:
# 원하는 데이터 뽑아오자
import sys
import requests
import base64
import json
import logging
import pymysql

host = data[0]
port = int(data[1])
username = "kangyeol"
database = data[2]
password = data[3]
client_id = data[4]
client_secret = data[5]


def main():
    
    
    
    try:
        conn = pymysql.connect(host, user=username, passwd=password, db=database,\
                             port=port, use_unicode=True, charset="utf8")
        cursor = conn.cursor()
    except:
        logging.error("could not connect to RDS")
        sys.exit(1)
        
    headers = get_headers(client_id, client_secret)
    
    ## Spotify Search API
    params = {
        "q": "BTS",
        "type": "artist",
        "limit": "5"
    }
    
    r = requests.get("https://api.spotify.com/v1/search", params=params, headers=headers)
    
    # json을 통해서 스트링 형태로 들어온다.
    raw = json.loads(r.text)

    artist = {}
    artist_raw = raw["artists"]["items"][0]
    
    if artist_raw["name"] == params["q"]:
        artist.update({
            "id": artist_raw["id"],
            "name": artist_raw["name"],
            "followers": artist_raw["followers"]["total"],
            "popularity": artist_raw["popularity"],
            "url": artist_raw["external_urls"]["spotify"],
            "image_url": artist_raw["images"][0]["url"]
        })
    
    insert_row(cursor, artist, "artists")
    conn.commit()
    sys.exit(0)
    
    


In [14]:
main()

SystemExit: 0

### 500명의 artist list를 내가만든 data table에 가져오자

In [27]:
# 원하는 데이터 뽑아오자
import sys
import requests
import base64
import json
import logging
import pymysql
import csv

host = data[0]
port = int(data[1])
username = "kangyeol"
database = data[2]
password = data[3]
client_id = data[4]
client_secret = data[5]


def main():
    
    
    
    try:
        conn = pymysql.connect(host, user=username, passwd=password, db=database,\
                             port=port, use_unicode=True, charset="utf8")
        cursor = conn.cursor()
    except:
        logging.error("could not connect to RDS")
        sys.exit(1)
        
    headers = get_headers(client_id, client_secret)
    
    ## Spotify Search API
    artists = []
    with open("../artist_list 복사본.csv")  as f:
        raw = csv.reader(f)
        for row in raw:
            artists.append(row[0])
    print(artists)

    for a in artists:
        params = {
            "q": a,
            "type": "artist",
            "limit": "1"        
        }
    
        r = requests.get("https://api.spotify.com/v1/search", params=params, headers=headers)
        raw = json.loads(r.text)
        
        artist = {}
        
        #에러 방지해주기 위해
        try:
            if raw["artists"]["items"][0]["name"] == params["q"]:
                
                artist.update(
                    {
                        "id": raw["artists"]["items"][0]["id"],
                        "name":raw["artists"]["items"][0]["name"],
                        "followers":raw["artists"]["items"][0]["followers"]["total"],
                        "popularity":raw["artists"]["items"][0]["popularity"],
                        "url": raw["artists"]["items"][0]["external_urls"]["spotify"],
                        "image_url": raw["artists"]["items"][0]["images"][0]["url"],
                    }
                )
                insert_row(cursor, artist, "artists")
                
        except:
            logging.error("NO ITEMS FROM SEARCH API")
            # 에러가 나도 지속적으로 해라
            continue
        

        
    conn.commit()
    sys.exit(0)
    
    r = requests.get("https://api.spotify.com/v1/search", params=params, headers=headers)
    
    # json을 통해서 스트링 형태로 들어온다.
    raw = json.loads(r.text)

    artist = {}
    artist_raw = raw["artists"]["items"][0]
    
    if artist_raw["name"] == params["q"]:
        artist.update({
            "id": artist_raw["id"],
            "name": artist_raw["name"],
            "followers": artist_raw["followers"]["total"],
            "popularity": artist_raw["popularity"],
            "url": artist_raw["external_urls"]["spotify"],
            "image_url": artist_raw["images"][0]["url"]
        })
    
    insert_row(cursor, artist, "artists")
    conn.commit()
    sys.exit(0)
    
    


In [28]:
main()

['\ufeff10cc', '2Pac', '50 Cent', 'A Thousand Horses', 'ABBA', 'ABC', 'Aerosmith', 'Agnetha Fältskog', 'Alan Jackson', 'Albert King', 'Alice Cooper', 'Alison Krauss', 'The All-American Rejects', 'The Allman Brothers Band', 'Amy Winehouse', 'Andre Rieu', 'Andrea Bocelli', 'Andrew W.K.', 'Anthrax', 'Antonio Carlos Jobim', 'Apache Indian', 'Arcade Fire', 'Ariana Grande', 'Arrested Development', 'Ashley Campbell', 'Astrud Gilberto', 'Aswad', 'Atlanta Rhythm Section', 'Audioslave', 'B.B. King', 'Badfinger', 'The Band', 'Barclay James Harvest', 'Barry White', 'The Beach Boys', 'Beastie Boys', 'The Beatles', 'Beck', 'Bee Gees', 'Belinda Carlisle', 'Ben Harper', 'Ben Howard', 'Benny Andersson', 'Big Country', 'Big Star', 'Bill Evans', 'Billie Holiday', 'Billy Currington', 'Billy Fury', 'Billy Preston', 'Björk', 'Black Eyed Peas', 'Black Sabbath', 'Black Uhuru', 'Blind Faith', 'Blink-182', 'Blondie', 'Blue Cheer', 'Bo Diddley', 'Bob Dylan', 'Bob Marley', 'Bon Jovi', 'Bonnie Raitt', 'Booker T', 

ERROR:root:NO ITEMS FROM SEARCH API
ERROR:root:NO ITEMS FROM SEARCH API
ERROR:root:NO ITEMS FROM SEARCH API
ERROR:root:NO ITEMS FROM SEARCH API
ERROR:root:NO ITEMS FROM SEARCH API
ERROR:root:NO ITEMS FROM SEARCH API
ERROR:root:NO ITEMS FROM SEARCH API


SystemExit: 0